In [1]:
import numpy as np
import pandas as pd
import pandas.util.testing as tm
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
plt.style.use('seaborn')
plotColors = (list(mcolors.TABLEAU_COLORS)+list(mcolors.TABLEAU_COLORS)+list(mcolors.TABLEAU_COLORS))[:23]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
!pip install impyute

In [3]:
path = '/content/drive/My Drive/Colab Notebooks/PA_1/Guava_tree/clean_csv_file.csv'
df_1= pd.read_csv(path)

In [4]:
df_1.columns

Index(['Unnamed: 0', 'Date', 'Temp_H', 'Temp_avg', 'Temp_L', 'Dew_H',
       'Dew_avg', 'Dew_L', 'Humidity_H', 'Humidity_avg', 'Humidity_L',
       'Pressure_H', 'Pressure_avg', 'Pressure_L', 'Visibility_H',
       'Visibility_avg', 'Visibility_L', 'Wind_H', 'Wind_avg', 'Gust_Speed_H',
       'Precip_sum', 'Events', 'Month'],
      dtype='object')

In [5]:
df_1.drop(columns=['Unnamed: 0',"Gust_Speed_H"],inplace = True)

In [6]:
df_1['Events'].value_counts()

        339
Rain     21
Fog      15
fog       9
rain      5
Name: Events, dtype: int64

In [7]:
df_1['Events']= df_1['Events'].replace(np.nan, 'neutral')
df_1['Events']= df_1['Events'].replace('Fog', 'fog')
df_1['Events']= df_1['Events'].replace('Rain', 'rain')

In [8]:
df_1.rename(columns={"Date": "Day"},inplace=True)

In [9]:
one_hot = pd.get_dummies(df_1['Events'])
# Drop column B as it is now encoded
df_1 = df_1.drop('Events',axis = 1)
# Join the encoded df
df_1 = df_1.join(one_hot)

In [10]:
df_1.columns

Index(['Day', 'Temp_H', 'Temp_avg', 'Temp_L', 'Dew_H', 'Dew_avg', 'Dew_L',
       'Humidity_H', 'Humidity_avg', 'Humidity_L', 'Pressure_H',
       'Pressure_avg', 'Pressure_L', 'Visibility_H', 'Visibility_avg',
       'Visibility_L', 'Wind_H', 'Wind_avg', 'Precip_sum', 'Month', 'fog',
       'neutral', 'rain', '  '],
      dtype='object')

In [11]:
path = '/content/drive/My Drive/Colab Notebooks/PA_1/Guava_tree/guava_dataset.xlsx'
df_2 = pd.read_excel(path,sheet_name='population', header = [1,2])

In [12]:
cols = df_2.columns.to_list()
cols = [','.join(y).replace(' ', '')  for y in cols] 

In [13]:
df_2.columns = cols

In [14]:
#df_2.rename(columns={"Date,Unnamed:0_level_1": "Date"},inplace=True)

In [15]:
df_2.head()

,"Date,Unnamed:0_level_1","mealybugspecies,Ferrisiavirgata","mealybugspecies,Iceryaseychellarum","mealybugspecies,Iceryapurchasi","mealybugspecies,planococcuscitri","Predators,Scymnussyriacus","Predators,Cydoniavicina","Predators,Chrysoperlacarnea","Predators,Rodalia","Parasitoidattackpredators,Homalotylusvicinus","Parasitoidattackpredators,Homalotyloidea","Primaryparasitoids,Leptomastix","Primaryparasitoids,Leptomastidae","Primaryparasitoids,Gyranusoideaindica","Primaryparasitoids,Aenasius","Hyperparasitoid,Chartocerussubaeneus"
0,2014-07-01 00:00:00,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,21/1/2014,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2014-04-02 00:00:00,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,18/2/2014,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2014-04-03 00:00:00,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
df_2.drop(columns =['Date,Unnamed:0_level_1'],inplace = True)

In [17]:
dataset = df_1.join(df_2, lsuffix='_caller', rsuffix='_other')

In [18]:
dataset.columns

Index(['Day', 'Temp_H', 'Temp_avg', 'Temp_L', 'Dew_H', 'Dew_avg', 'Dew_L',
       'Humidity_H', 'Humidity_avg', 'Humidity_L', 'Pressure_H',
       'Pressure_avg', 'Pressure_L', 'Visibility_H', 'Visibility_avg',
       'Visibility_L', 'Wind_H', 'Wind_avg', 'Precip_sum', 'Month', 'fog',
       'neutral', 'rain', '  ', 'mealybugspecies,Ferrisiavirgata',
       'mealybugspecies,Iceryaseychellarum', 'mealybugspecies,Iceryapurchasi',
       'mealybugspecies,planococcuscitri', 'Predators,Scymnussyriacus',
       'Predators,Cydoniavicina', 'Predators,Chrysoperlacarnea',
       'Predators,Rodalia', 'Parasitoidattackpredators,Homalotylusvicinus',
       'Parasitoidattackpredators,Homalotyloidea',
       'Primaryparasitoids,Leptomastix', 'Primaryparasitoids,Leptomastidae',
       'Primaryparasitoids,Gyranusoideaindica', 'Primaryparasitoids,Aenasius',
       'Hyperparasitoid,Chartocerussubaeneus'],
      dtype='object')

In [19]:
un_imputed_1 = dataset.copy()

In [20]:
un_imputed_2 = dataset.copy()

In [21]:
un_imputed_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 39 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Day                                           730 non-null    int64  
 1   Temp_H                                        730 non-null    int64  
 2   Temp_avg                                      730 non-null    int64  
 3   Temp_L                                        730 non-null    int64  
 4   Dew_H                                         730 non-null    int64  
 5   Dew_avg                                       730 non-null    int64  
 6   Dew_L                                         730 non-null    int64  
 7   Humidity_H                                    730 non-null    int64  
 8   Humidity_avg                                  730 non-null    int64  
 9   Humidity_L                                    730 non-null    int

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from math import sqrt
#Impute the values using scikit-learn SimpleImpute Class
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer( strategy='mean') #for median imputation replace 'mean' with 'median'
imp_mean.fit(un_imputed_1)
imputed_df = imp_mean.transform(un_imputed_1)
imputed_1_df = pd.DataFrame(imputed_df,columns=un_imputed_1.columns)
imputed_1_df.head(2)

,Day,Temp_H,Temp_avg,Temp_L,Dew_H,Dew_avg,Dew_L,Humidity_H,Humidity_avg,Humidity_L,Pressure_H,Pressure_avg,Pressure_L,Visibility_H,Visibility_avg,Visibility_L,Wind_H,Wind_avg,Precip_sum,Month,fog,neutral,rain,,"mealybugspecies,Ferrisiavirgata","mealybugspecies,Iceryaseychellarum","mealybugspecies,Iceryapurchasi","mealybugspecies,planococcuscitri","Predators,Scymnussyriacus","Predators,Cydoniavicina","Predators,Chrysoperlacarnea","Predators,Rodalia","Parasitoidattackpredators,Homalotylusvicinus","Parasitoidattackpredators,Homalotyloidea","Primaryparasitoids,Leptomastix","Primaryparasitoids,Leptomastidae","Primaryparasitoids,Gyranusoideaindica","Primaryparasitoids,Aenasius","Hyperparasitoid,Chartocerussubaeneus"
0,1.0,19.0,14.0,11.0,11.0,6.0,0.0,82.0,59.0,22.0,1021.0,1019.0,1018.0,10.0,4.0,1.0,14.0,8.0,0.0,1.0,0.0,0.0,0.0,1.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,21.0,16.0,11.0,9.0,6.0,1.0,67.0,53.0,38.0,1024.0,1023.0,1021.0,10.0,6.0,3.0,40.0,13.0,0.0,1.0,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
imputed_1_df.to_excel('/content/drive/My Drive/Colab Notebooks/PA_1/Guava_tree/impued_data_mean.xlsx')

In [24]:
imputed_1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 39 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Day                                           730 non-null    float64
 1   Temp_H                                        730 non-null    float64
 2   Temp_avg                                      730 non-null    float64
 3   Temp_L                                        730 non-null    float64
 4   Dew_H                                         730 non-null    float64
 5   Dew_avg                                       730 non-null    float64
 6   Dew_L                                         730 non-null    float64
 7   Humidity_H                                    730 non-null    float64
 8   Humidity_avg                                  730 non-null    float64
 9   Humidity_L                                    730 non-null    flo

In [25]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000) #Increase the recursion limit of the OS
# start the KNN training
imputed_2=fast_knn(un_imputed_2.values, k=30)

In [26]:
imputed_2_df = pd.DataFrame(data = imputed_2,columns=un_imputed_2.columns)

In [27]:
imputed_2_df.head(2)

,Day,Temp_H,Temp_avg,Temp_L,Dew_H,Dew_avg,Dew_L,Humidity_H,Humidity_avg,Humidity_L,Pressure_H,Pressure_avg,Pressure_L,Visibility_H,Visibility_avg,Visibility_L,Wind_H,Wind_avg,Precip_sum,Month,fog,neutral,rain,,"mealybugspecies,Ferrisiavirgata","mealybugspecies,Iceryaseychellarum","mealybugspecies,Iceryapurchasi","mealybugspecies,planococcuscitri","Predators,Scymnussyriacus","Predators,Cydoniavicina","Predators,Chrysoperlacarnea","Predators,Rodalia","Parasitoidattackpredators,Homalotylusvicinus","Parasitoidattackpredators,Homalotyloidea","Primaryparasitoids,Leptomastix","Primaryparasitoids,Leptomastidae","Primaryparasitoids,Gyranusoideaindica","Primaryparasitoids,Aenasius","Hyperparasitoid,Chartocerussubaeneus"
0,1.0,19.0,14.0,11.0,11.0,6.0,0.0,82.0,59.0,22.0,1021.0,1019.0,1018.0,10.0,4.0,1.0,14.0,8.0,0.0,1.0,0.0,0.0,0.0,1.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,21.0,16.0,11.0,9.0,6.0,1.0,67.0,53.0,38.0,1024.0,1023.0,1021.0,10.0,6.0,3.0,40.0,13.0,0.0,1.0,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
imputed_2_df.to_excel('/content/drive/My Drive/Colab Notebooks/PA_1/Guava_tree/impued_data_KNN.xlsx')

In [29]:
X_cols =   ['Day',
            'Temp_H',
            'Temp_avg',
            'Temp_L',
            'Dew_H',
            'Dew_avg',
            'Dew_L',
            'Humidity_H',
            'Humidity_avg',
            'Humidity_L',
            'Pressure_H',
            'Pressure_avg',
            'Pressure_L',
            'Visibility_H',
            'Visibility_avg',
            'Visibility_L',
            'Wind_H',
            'Wind_avg',
            'Precip_sum',
            'fog',
            'Month',
            'neutral',
            'rain']

X = imputed_2_df[X_cols].values

In [30]:
Y_cols = ['mealybugspecies,Ferrisiavirgata',
          'mealybugspecies,Iceryaseychellarum',
          'mealybugspecies,Iceryapurchasi',
          'mealybugspecies,planococcuscitri',
          'Predators,Scymnussyriacus',
          'Predators,Cydoniavicina',
          'Predators,Chrysoperlacarnea',
          'Predators,Rodalia',
          'Parasitoidattackpredators,Homalotylusvicinus',
          'Parasitoidattackpredators,Homalotyloidea',
          'Primaryparasitoids,Leptomastix',
          'Primaryparasitoids,Leptomastidae',
          'Primaryparasitoids,Gyranusoideaindica',
          'Primaryparasitoids,Aenasius',
          'Hyperparasitoid,Chartocerussubaeneus']

y_tot_1 = imputed_1_df[Y_cols].values

In [31]:
y_tot_2 = imputed_2_df[Y_cols].values

In [32]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
sc_y = StandardScaler() 

X_new = sc_x.fit_transform(X)

In [33]:


from math import sqrt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV

In [34]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

clf = MultiOutputRegressor(Ridge(random_state=123)).fit(X, y_tot_1)
clf.predict(X[[0]])

array([[1.27697512e+03, 3.82644577e+01, 4.64017103e+00, 2.59896581e+01,
        5.00280156e+01, 2.03429944e+00, 1.14412704e+00, 6.43612066e-01,
        2.74958349e+01, 3.37152193e-01, 5.10193274e+00, 4.21099610e-01,
        1.77604434e+00, 9.12843520e-01, 1.60001088e+02]])

In [35]:
clf_2 = MultiOutputRegressor(Ridge(random_state=123)).fit(X, y_tot_2)
clf_2.predict(X[[0]])

array([[1.27697512e+03, 3.82644577e+01, 4.64017103e+00, 2.59896581e+01,
        5.00280156e+01, 2.03429944e+00, 1.14412704e+00, 6.43612066e-01,
        2.74958349e+01, 3.37152193e-01, 5.10193274e+00, 4.21099610e-01,
        1.77604434e+00, 9.12843520e-01, 1.60001088e+02]])

In [36]:
data_dmatrix_1 = xgb.DMatrix(data=X,label=y_tot_1)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


In [38]:
objective = "reg:linear"
seed = 100
n_estimators = 100
learning_rate = 0.1
gamma = 0.1
subsample = 0.8
colsample_bytree =1
reg_alpha = 0.5
reg_lambda = 0.5
silent = False
max_depth = 6
min_child_weight = 0
subsample = 1.0
rmse=[]
rrmse=[]
parameters = {}
parameters['objective'] = objective
parameters['seed'] = seed
parameters['n_estimators'] = n_estimators
parameters['learning_rate'] = learning_rate
parameters['gamma'] = gamma
parameters['colsample_bytree'] = colsample_bytree
parameters['reg_alpha'] = reg_alpha
parameters['reg_lambda'] = reg_lambda
parameters['silent'] = silent

X_train, X_test, y_train, y_test = train_test_split(X, y_tot_2, test_size=0.2, random_state=123)
xg_reg_multioutputregressor = MultiOutputRegressor(xgb.XGBRegressor(objective = objective,
                                        seed = seed,
                                        n_estimators = n_estimators,
                                        learning_rate = learning_rate,
                                        gamma = gamma,
                                        subsample = subsample,
                                        colsample_bytree = colsample_bytree,
                                        reg_alpha = reg_alpha,
                                        reg_lambda = reg_lambda,
                                        silent = silent)).fit(X_train,y_train)
   

  
preds = xg_reg_multioutputregressor.predict(X_test[:])     
rmse= mean_squared_error(y_test, preds,squared=False, multioutput='raw_values')
#rmse= np.sqrt(mean_squared_error(y_test,preds, multioutput='raw_values'))*100
rrmse = np.sqrt(mean_squared_error(y_test, preds, multioutput='raw_values'))*100/np.mean(y_test)


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


[19:47:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [ ]:
#preds = xg_reg_multioutputregressor.predict(X_test[:])     
#preds

In [39]:
rmse

array([4.32890137e+04, 1.36038959e+02, 2.21334509e+00, 7.86891928e+02,
       1.45284089e+02, 1.32388887e+00, 9.22503890e-02, 4.59737139e-02,
       5.35827928e+01, 1.11277284e-02, 9.72130628e+00, 2.52813069e-02,
       3.83233816e+00, 1.20396788e-01, 4.25932506e+03])

In [40]:
rrmse

array([2.25769821e+02, 1.26563563e+01, 1.61436454e+00, 3.04392950e+01,
       1.30793473e+01, 1.24854100e+00, 3.29580226e-01, 2.32665419e-01,
       7.94309427e+00, 1.14467001e-01, 3.38329103e+00, 1.72534840e-01,
       2.12426628e+00, 3.76516917e-01, 7.08186162e+01])

In [ ]:
y_test_predict = dict()
y_rrmse = dict()
y_test_predict['xgb'] = xg_reg_multioutputregressor.predict(X_test)   # Make predictions and save it in dict under key: name
y_rrmse['rrmse_xgb'] = np.sqrt(mean_squared_error(y_test, y_test_predict['xgb']))/np.mean(y_test)

In [ ]:
y_rrmse